In [11]:
!pip install fastapi uvicorn faster-whisper pydub psutil pyngrok nest_asyncio tenacity torch deep-translator requests
from google.colab import userdata
GITHUB_TOKEN = userdata.get("GIT_TOKEN")
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
import os
import psutil
import signal
import time
import re
import requests
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse
from faster_whisper import WhisperModel
from pydub import AudioSegment
from io import BytesIO
import torch
import logging
import uvicorn
from pyngrok import ngrok
import nest_asyncio
from tenacity import retry, stop_after_attempt, wait_fixed
from deep_translator import GoogleTranslator

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Retrieve secrets from environment variables
import os

GIST_ID = "e1b9e5a9d7167ab2458405c38a2803c2"
GIST_FILENAME = "ngrok_public_url.txt"
GIST_DESCRIPTION = "Ngrok Public URL for FastAPI Server"

# Check if secrets are set
if not GIT_TOKEN or not NGROK_AUTH_TOKEN:
    raise ValueError("GITHUB_TOKEN or NGROK_AUTH_TOKEN not set in environment variables")

# Function to free port 8000
def free_port(port=8000):
    try:
        for proc in psutil.process_iter(['pid', 'name']):
            for conn in proc.net_connections(kind='inet'):
                if conn.laddr.port == port:
                    logger.info(f"Terminating process {proc.pid} using port {port}")
                    os.kill(proc.pid, signal.SIGTERM)
                    proc.wait(timeout=5)
    except Exception as e:
        logger.warning(f"Error freeing port {port}: {str(e)}")

# Function to create or update a GitHub Gist
@retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
def update_github_gist(public_url: str) -> str:
    global GIST_ID
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    gist_data = {
        "description": GIST_DESCRIPTION,
        "public": False,
        "files": {
            GIST_FILENAME: {
                "content": public_url
            }
        }
    }

    try:
        if GIST_ID:
            # Update existing Gist
            response = requests.patch(
                f"https://api.github.com/gists/{GIST_ID}",
                headers=headers,
                json=gist_data
            )
            if response.status_code == 200:
                logger.info(f"Successfully updated Gist {GIST_ID} with URL: {public_url}")
                return GIST_ID
            else:
                logger.error(f"Failed to update Gist {GIST_ID}: {response.text}")
                raise Exception(f"Failed to update Gist: {response.text}")
        else:
            # Create new Gist
            response = requests.post(
                "https://api.github.com/gists",
                headers=headers,
                json=gist_data
            )
            if response.status_code == 201:
                GIST_ID = response.json()['id']
                logger.info(f"Created new Gist {GIST_ID} with URL: {public_url}")
                return GIST_ID
            else:
                logger.error(f"Failed to create Gist: {response.text}")
                raise Exception(f"Failed to create Gist: {response.text}")
    except Exception as e:
        logger.error(f"Error updating/creating Gist: {str(e)}")
        raise

# Function to start ngrok tunnel
def start_ngrok_tunnel(port=8000):
    try:
        tunnel = ngrok.connect(port, bind_tls=True)
        public_url = tunnel.public_url
        logger.info(f"ngrok tunnel created at {public_url}")
        return public_url
    except Exception as e:
        logger.error(f"Failed to start ngrok tunnel: {str(e)}")
        raise

# Initialize FastAPI app
app = FastAPI()

# Load Whisper model
model = None
try:
    model = WhisperModel("large", device="cuda", compute_type="float16")
    logger.info("Whisper model loaded on GPU")
except Exception as e:
    logger.warning(f"Failed to load Whisper model on GPU: {str(e)}. Falling back to CPU.")
    try:
        model = WhisperModel("large", device="cpu", compute_type="int8")
        logger.info("Whisper model loaded on CPU")
    except Exception as cpu_e:
        logger.error(f"Failed to load Whisper model on CPU: {str(cpu_e)}")
        raise RuntimeError(f"Failed to load Whisper model on both GPU and CPU: {str(cpu_e)}")

def clean_transcription(text):
    """Clean the transcription by removing consecutive duplicates"""
    sentences = [s.strip() for s in re.split(r'[.!?]', text) if s.strip()]
    unique_sentences = []
    prev_sentence = None

    for sentence in sentences:
        if sentence != prev_sentence:
            unique_sentences.append(sentence)
            prev_sentence = sentence

    processed_sentences = []
    for sentence in unique_sentences:
        words = sentence.split()
        unique_words = []
        prev_word = None
        for word in words:
            if word != prev_word:
                unique_words.append(word)
                prev_word = word
        processed_sentences.append(' '.join(unique_words))

    cleaned_text = '. '.join(processed_sentences)
    if text.endswith('.'):
        cleaned_text += '.'
    elif text.endswith('?'):
        cleaned_text += '?'
    elif text.endswith('!'):
        cleaned_text += '!'

    return cleaned_text

def translate_to_language(text_or_dict, source_lang, target_lang):
    lang_codes = {
        'en': 'en', 'hi': 'hi', 'bn': 'bn', 'te': 'te', 'mr': 'mr',
        'ta': 'ta', 'ur': 'ur', 'gu': 'gu', 'kn': 'kn', 'ml': 'ml',
        'pa': 'pa', 'or': 'or', 'as': 'as', 'ne': 'ne'
    }
    malayalam_reverse_map = {
            'ആശയവുമില്ല': 'ഐഡിയയുമില്ല',
            'എന്ന് ': 'നിന്നും ',
            'ചെയ്യാൻ': 'ചെയ്യാവോ',
            'പരിഹാരങ്ങളും': 'സൊല്യൂഷൻസ്',
            'പരിശീലനാർത്ഥി': 'ട്രെയിനീ',
            'തീരുമാനം': 'ഡിസിഷൻ',
            'ഉദ്യോഗം': 'ജോബ്',
            'പദ്ധതി': 'പ്രൊജക്ട്',
            'സമസ്യ': 'പ്രോബ്ലം',
            'സഹായം': 'ഹെൽപ്പ്',
            'ഉദാഹരണം': 'എക്സാംപിൾ',
            'വിവരണം': 'ഡിസ്‌ക്രിപ്ഷൻ',
            'വിലയിരുത്തൽ': 'ഇവാലുവേഷൻ',
            'പരീക്ഷണം': 'ടെസ്റ്റ്',
            'പരീക്ഷ': 'എക്സാം',
            'പഠനം': 'സ്റ്റഡി',
            'വിദ്യാർത്ഥി': 'സ്റ്റുഡന്റ്',
            'വ്യാപാരം': 'ബിസിനസ്',
            'തൊഴിലാളി': 'എംപ്ലോയി',
            'ഉദ്യോഗസ്ഥൻ': 'സ്റ്റാഫ്',
            'കൂടിയാലോചന': 'മീറ്റിംഗ്',
            'ആരംഭം': 'സ്റ്റാർട്ട്',
            'അവസാനം': 'എൻഡ്',
            'സംശയം': 'ഡൗട്ട്',
            'പിന്തുണ': 'സപ്പോർട്ട്',
            'ആശ്വാസം': 'റിലീഫ്',
            'താല്പര്യം': 'ഇന്ററസ്റ്റ്',
            'പ്രതിഫലം': 'റിവാർഡ്',
            'പരിശോധന': 'ചെക്ക്',
            'ഉപകരണങ്ങൾ': 'ടൂൾസ്',
            'തെളിവ്': 'പ്രൂഫ്',
            'അനുമതി': 'പർമിഷൻ',
            'ലക്ഷ്യം': 'ഗോൾ',
            'വൈഭവം': 'ഗ്ലാമർ',
            'സംരക്ഷണം': 'സെക്യൂരിറ്റി',
            'നയം': 'പോളിസി',
            'പരിധി': 'ലിമിറ്റ്',
            'പരീക്ഷണഘട്ടം': 'പൈലറ്റ്',
            'സവിശേഷത': 'ഫീച്ചർ',
            'തികഞ്ഞത്': 'ഓപ്റ്റിമൽ',
            'മാറ്റം': 'ചേഞ്ച്',
            'ഉപഭോക്താവ്': 'യൂസർ',
            'പ്രകടനം': 'പെർഫോമൻസ്',
            'വിശ്വാസം': 'ട്രസ്റ്റ്',
            'വൈഭവം': 'ഗ്രാൻഡർ',
            'വിഫലം': 'ഫെയിൽഡ്',
            'വിജയം': 'സക്സസ്',
            'നിര്‍ണ്ണയം': 'ജഡ്ജ്മെന്റ്',
            'പ്രതീക്ഷ': 'എക്‌സ്‌പെക്ടേഷൻ',
            'അവസരം': 'ഓപ്പർച്യൂണിറ്റി',
            'തീരുവിൽ': 'ഫൈനലായി',
            'കണ്ണോട്ട്': 'ഫോക്കസ്',
            'തെളിവാക്കുക': 'വെരിഫൈ ചെയ്യുക',
            'മാറ്റിവെക്കുക': 'റീഷെഡ്യൂൾ ചെയ്യുക',
            'നൽകുക': 'പ്രൊവൈഡ് ചെയ്യുക',
            'പരിഷ്കരണം': 'അപ്പ്‌ഡേറ്റ്',
            'തിരഞ്ഞെടുത്തത്': 'സെലെക്ട് ചെയ്തത്',
            'തിരഞ്ഞെടുപ്പ്': 'ഓപ്ഷൻ',
            'സ്ഥിരീകരണം': 'കോൺഫർമേഷൻ',
            'ചിട്ട': 'പ്ലാൻ',
            'പരാമർശം': 'റഫറൻസ്',
            'മാറ്റം': 'മോഡിഫിക്കേഷൻ',
            'അഭിപ്രായം': 'ഫീഡ്‌ബാക്ക്',
            'കാഴ്ചപ്പാട്': 'വ്യൂ',
            'പിന്തുടരുക': 'ഫോളോ ചെയ്യുക',
            'ഭരണസംവിധാനം': 'മാനേജ്മെന്റ്',
            'അനുഭവം': 'എക്സ്പീരിയൻസ്',
            'ഉണ്ടോ': 'വേണോ',
            'തന്ത്രശാസ്ത്ര സ്ഥാപനം': 'ഐടി ഫിർം',
            'നയം': 'പോളിസി',
            'പരിഗണന': 'കൺസിഡറേഷൻ',
            'നിബന്ധനകൾ': 'ടേംസ്',
            'സമിതി': 'കമ്മിറ്റി',
            'നിക്ഷേപം': 'ഇൻവെസ്റ്റ്‌മെന്റ്',
            'സംരംഭം': 'സ്റ്റാർട്ടപ്പ്',
            'തൊഴിൽ': 'ജോബ്',
            'പരിശീലനം': 'ട്രെയിനിംഗ്',
            'വായ്പ': 'ലോൺ',
            'തരംഗം': 'ട്രെൻഡ്',
            'നിക്ഷേപകന്‍': 'ഇൻവെസ്റ്റർ',
            'പദ്ധതി': 'പ്രൊജക്ട്',
            'അവകാശം': 'റൈറ്റ്‌സ്',
            'സുരക്ഷ': 'സെക്യൂരിറ്റി',
            'അനുമതി': 'പർമിഷൻ',
            'നയതന്ത്രം': 'ഡിപ്ലോമസി',
            'വിനിമയം': 'ട്രാൻസാക്ഷൻ',
            'പരിഹാരങ്ങളും': 'സൊല്യൂഷൻസ്',
            'മീൻ/മെൻ ടെക്‌നോളജി പ്ലാറ്റ്ഫോം': 'മീൻ ആൻഡ് മെൻ സ്റ്റാക്ക്',
            'ആവശ്യ പ്രയോഗ നിർമ്മാണം': 'ആപ്ലിക്കേഷൻ ഡെവലപ്മെന്റ്',
            'അനുഭവ സർട്ടിഫിക്കറ്റ്': 'എക്സ്പീരിയൻസ് സർട്ടിഫിക്കറ്റ്',
            'തിരഞ്ഞെടുത്ത പ്രക്രിയ': 'സ്ക്രീനിംഗ് പ്രോസസ്സ്',
            'ഇന്റർനെറ്റ് സുരക്ഷാ': 'സൈബർ സെക്യൂരിറ്റി',
            'ഡാറ്റാ ശാസ്ത്രം': 'ഡേറ്റ സയൻസ്',
            'ജീവന്ത പദ്ധതികൾ': 'ലൈവ് പ്രൊജക്ട്സ്',
            'ജീവന്ത പരിശീലനം': 'ലൈവ് ട്രെയിനിംഗ്',
            'പണമേറ്റ് പരിശീലനം': 'പെയ്ഡ് ഇന്റേൺഷിപ്പ്',
            'സുരക്ഷാ പരിഹാരങ്ങൾ': 'സെക്യൂരിറ്റി സൊല്യൂഷൻസ്',
            'പൂർത്തീകരണ സർട്ടിഫിക്കറ്റ്': 'സർട്ടിഫിക്കറ്റ് ഓഫ് കമ്പ്ലീഷൻ',
            'കൂടുതൽ വിവരം': 'മോർ ഇൻഫർമേഷൻ',
            'അവശ്യമില്ല': 'നോ നീഡ്',
            'സാധ്യമല്ല': 'നോട്ട് പൊസിബിൾ',
            'അറിയിപ്പ് ലഭിച്ചു': 'നോട്ടിഫിക്കേഷൻ റിസീവ്ഡ്',
            'ആറായിരം': 'സിക്സ് തൗസൻഡ്',
            'സോഫ്റ്റ്വെയറിൽ': 'സോഫ്റ്റ്‌വെയർ',
            'പരിഹാരങ്ങൾ': 'സൊല്യൂഷൻസ്',
            'പരിഹാരം': 'സൊല്യൂഷൻ',
            'മൃദുസ്ഥിതി പ്രയോഗം': 'സോഫ്റ്റ്‌വെയർ',
            'പരിശീലനം': 'ഇന്റേൺഷിപ്പ്',
            'സാക്ഷ്യപത്രം': 'സർട്ടിഫിക്കറ്റ്',
            'അടിസ്ഥാനഘടന': 'ഫ്രെയിംവർക്ക്',
            'ചോദ്യോത്തരസമരം': 'ഇന്റർവ്യൂ',
            'തൊഴിൽനിയമനം': 'പ്ലേസ്മെന്റ്',
            'ഭരണസംവിധാനം': 'മാനേജ്മെന്റ്',
            'അനുഭവം': 'എക്സ്പീരിയൻസ്',
            'ഉണ്ടോ': 'വേണോ',
            'തന്ത്രശാസ്ത്ര സ്ഥാപനം': 'ഐടി ഫിർം',
            'പരിഹാരവുമായി': 'സൊലൂഷൻസ്',
            'വാട്ട്‌സാപ്പ്': 'വാട്ട്‌സാപ്പ്',
            'സുരക്ഷാ': 'സെക്യൂരിറ്റി',
            'സ്ഥാപനം': 'കമ്പനി',
            'പദ്ധതി': 'പ്രൊജക്ട്',
            'ബന്ധപ്പെടുന്നു': 'വിളിക്കുന്നു',
            'ഗൂഗിൾ': 'ഗൂഗിൾ',
            'പരിഹാരവും': 'സൊലൂഷൻസ്',
            'കൃത്രിമ ബുദ്ധിമത്താ': 'എഐ',
            'മെഷീൻ ലേണിംഗ്': 'എംഎൽ',
            'മാനവ വിഭവശേഷി': 'എച് ആർ',
            'യോഗ്യത': 'ക്വാളിഫിക്കേഷൻ',
            'നികുതി': 'ടാക്സ്'
        }
    try:
        if isinstance(text_or_dict, dict):
            text = text_or_dict.get('raw_transcription', '')
            if not text:
                logger.warning("No raw_transcription found in input dictionary")
                return {'translated_text': ''}
        else:
            text = text_or_dict
            if not isinstance(text, str):
                logger.warning(f"Input text_or_dict is not a string or dict: {type(text_or_dict)}")
                text = str(text_or_dict)
        text = text.strip()
        if not text:
            logger.warning("Input text is empty after stripping")
            return {'translated_text': ''}
        text = re.sub(r'[^\w\s.,!?]', '', text)
        if source_lang not in lang_codes or target_lang not in lang_codes:
            logger.error(f"Unsupported language: source={source_lang}, target={target_lang}")
            return {'translated_text': '', 'error': f"Unsupported language: source={source_lang}, target={target_lang}"}
        source_code = lang_codes.get(source_lang, 'en')
        target_code = lang_codes.get(target_lang, 'ml')
        logger.info(f"Translating from {source_lang} ({source_code}) to {target_lang} ({target_code})")
        translator = GoogleTranslator(source=source_code, target=target_code)
        translated_text = translator.translate(text)
        if not translated_text:
            logger.warning("Translation returned empty result")
            return {'translated_text': ''}
        if target_lang == 'ml':
            for incorrect, correct in malayalam_reverse_map.items():
                translated_text = translated_text.replace(incorrect, correct)
        logger.debug(f"Translated text: {translated_text[:100]}...")
        return {'translated_text': translated_text}
    except Exception as e:
        logger.error(f"Translation failed: {str(e)}")
        return {'translated_text': '', 'error': f"Translation failed: {str(e)}"}

@app.post("/transcribe")
async def transcribe(
    audio: UploadFile = File(...),
    model_size: str = Form("small"),
    transcription_language: str = Form("en"),
    target_language: str = Form(None)
):
    try:
        audio_data = await audio.read()
        audio_buffer = BytesIO(audio_data)
        audio_segment = AudioSegment.from_file(audio_buffer, format="wav")
        wav_buffer = BytesIO()
        audio_segment.export(wav_buffer, format="wav")
        wav_buffer.seek(0)
        segments, info = model.transcribe(
            wav_buffer,
            language=transcription_language,
            task="transcribe",
            beam_size=1,
            vad_filter=True
        )

        raw_segments = [segment.text for segment in segments]
        raw_transcription = " ".join(raw_segments)
        raw_transcription = clean_transcription(raw_transcription)

        response = {
            "raw_transcription": raw_transcription,
            "audio_metadata": {
                "extension": audio.filename.split('.')[-1],
                "duration": len(audio_segment) / 1000,
                "emotion": "unknown"
            },
            "detected_language": info.language,
            "detected_language_name": {
                'en': 'English', 'hi': 'Hindi', 'bn': 'Bengali', 'te': 'Telugu', 'mr': 'Marathi',
                'ta': 'Tamil', 'ur': 'Urdu', 'gu': 'Ggujarati', 'kn': 'Kannada', 'ml': 'Malayalam',
                'pa': 'Punjabi', 'or': 'Odia', 'as': 'Assamese', 'ne': 'Nepali'
            }.get(info.language, info.language)
        }

        if target_language:
            translation_result = translate_to_language(
                {'raw_transcription': raw_transcription},
                source_lang=info.language,
                target_lang=target_language
            )
            response['translated_text'] = translation_result.get('translated_text', '')
            if 'error' in translation_result:
                response['translation_error'] = translation_result['error']

        return JSONResponse(response)
    except Exception as e:
        logger.error(f"Transcription failed: {str(e)}")
        return JSONResponse({"error": str(e)}, status_code=500)

# Start ngrok tunnel and server
if __name__ == "__main__":
    nest_asyncio.apply()
    try:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        free_port(8000)
        public_url = start_ngrok_tunnel(8000)
        print(f"Public URL: {public_url}")
        GIST_ID = update_github_gist(str(public_url))
        print(f"Updated Gist ID: {GIST_ID}")
        uvicorn.run(app, host="0.0.0.0", port=8000)
    except Exception as e:
        logger.error(f"Failed to start server: {str(e)}")
        print("Error: Could not start server. Please check the ngrok dashboard[](https://dashboard.ngrok.com/agents) to terminate existing sessions or upgrade your ngrok account.")
        raise

ValueError: GITHUB_TOKEN or NGROK_AUTH_TOKEN not set in environment variables